# RA-UCB for Discrete Weibull - Criteo Experiments


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import weibull_min

# Import Discrete Weibull RA-UCB functions
from Utils_weibull_discrete import (
    oracle_discrete_weibull,
    discrete_weibull_pmf,
    discrete_weibull_cdf,
    discrete_weibull_sample,
    compute_confidence_bounds_discrete_weibull_vec,
    select_allocation_discrete,
    update_estimates_discrete_weibull,
    compute_regret,
    simulate_one_round_discrete_weibull,
)

# Load Data

In [ ]:
# Load the exposures before click dataset (created from Criteo)
DATA_PATH = "./data/criteo_exposures_before_click.csv"
df = pd.read_csv(DATA_PATH)

print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(df.head())

# Select top K campaigns
K = 15
top_campaigns = df['campaign'].value_counts().head(K).index.tolist()
df_filtered = df[df['campaign'].isin(top_campaigns)].copy()

print(f"\nUsing top {K} campaigns: {top_campaigns}")
print(f"Filtered dataset size: {len(df_filtered)}")

# Estimate Discrete Weibull parameters for each campaign (arm)
# For discrete Weibull: P(X = k) = q^(k^β) - q^((k+1)^β)
# We estimate β (shape) and q (scale) from the observed data

def fit_discrete_weibull(data, beta_fixed=None):
    """
    Fit discrete Weibull parameters to data.
    If beta_fixed is provided, only estimate q.
    Uses method of moments / MLE approximation.
    """
    data = np.asarray(data)
    mean_obs = np.mean(data)
    var_obs = np.var(data)
    
    if beta_fixed is not None:
        beta = beta_fixed
    else:
        # Estimate beta from coefficient of variation
        # For discrete Weibull, higher beta = more concentrated
        cv = np.std(data) / (mean_obs + 1e-6)
        beta = max(0.5, min(3.0, 1.5 / (cv + 0.1)))
    
    # Estimate q using mean
    # E[X] ≈ Σ q^((k+1)^β) for small q
    # Approximate: q ≈ exp(-1/mean) for beta=1
    # More generally, use numerical search
    from scipy.optimize import minimize_scalar
    
    def neg_likelihood(q):
        if q <= 0.01 or q >= 0.99:
            return 1e10
        # Compute expected mean for this q
        expected_mean = 0
        for k in range(int(max(data)) + 10):
            expected_mean += q ** ((k + 1) ** beta)
            if q ** ((k + 1) ** beta) < 1e-10:
                break
        return (expected_mean - mean_obs) ** 2
    
    result = minimize_scalar(neg_likelihood, bounds=(0.01, 0.99), method='bounded')
    q = result.x
    
    return q, beta

# Estimate parameters for each campaign
campaign_params = {}
for campaign in top_campaigns:
    campaign_data = df_filtered[df_filtered['campaign'] == campaign]
    exposures = campaign_data['exposures_before_click'].values
    p_click = campaign_data['clicked'].mean()
    
    # Fit discrete Weibull to the exposure counts
    q_est, beta_est = fit_discrete_weibull(exposures, beta_fixed=1.5)  # Fix beta for now
    
    campaign_params[campaign] = {
        'p': p_click,
        'q': q_est,
        'beta': beta_est,
        'n_samples': len(campaign_data),
        'mean_exposures': np.mean(exposures),
    }

# Create parameter arrays
campaign_list = top_campaigns
p_true = np.array([campaign_params[c]['p'] for c in campaign_list])
q_true = np.array([campaign_params[c]['q'] for c in campaign_list])
beta_vec = np.array([campaign_params[c]['beta'] for c in campaign_list])

print("\n=== Estimated Parameters ===")
params_df = pd.DataFrame(campaign_params).T
print(params_df)

# Single Experiment - RA-UCB with Discrete Weibull

In [ ]:
# Simulation parameters
T = 2000  # Total rounds
B = 50    # Total budget (number of exposures to allocate)
D = 1.0   # Confidence parameter
method = "dp"  # Use dynamic programming for discrete optimization

print("=" * 70)
print(f"SIMULATION: T={T}, K={K}, B={B}")
print("=" * 70)

print("\nTrue parameters (unknown to algorithm):")
print(f"p_true = {p_true}")
print(f"q_true = {q_true}")
print(f"beta_vec (known) = {beta_vec}")

# Compute optimal allocation (oracle knows everything)
x_star = oracle_discrete_weibull(p_true, q_true, q_true, B, beta_vec, method=method)
x_star = np.round(x_star).astype(int)  # Ensure integer allocations
# Adjust to ensure sum = B
diff = B - np.sum(x_star)
if diff != 0:
    x_star[np.argmax(x_star)] += diff

print(f"\nx_star = {x_star}")
print(f"sum(x_star) = {np.sum(x_star)}")

# Compute expected rewards
expected_reward_opt = np.sum(p_true * (1 - q_true ** ((x_star + 1) ** beta_vec)))
x_uniform = np.full(K, B // K)
x_uniform[0] += B - np.sum(x_uniform)  # Adjust for rounding
expected_reward_uniform = np.sum(p_true * (1 - q_true ** ((x_uniform + 1) ** beta_vec)))

print(f"\nExpected reward (optimal): {expected_reward_opt:.4f}")
print(f"Expected reward (uniform): {expected_reward_uniform:.4f}")

# Initialize random generator
rng = np.random.default_rng(42)

# SINGLE EXPERIMENT
t1 = 0  # Reward time counter
init_fraction = 0.001
init_steps_per_arm = max(1, int(init_fraction * T / K))

print(f"\ninit_steps_per_arm = {init_steps_per_arm}")

# Estimation Variables
n_rounds = int(T / K) + 1
n = np.zeros((n_rounds, K))
sum_obs = np.zeros((n_rounds, K))
q_hat = np.zeros((n_rounds, K))
p_hat = np.zeros((n_rounds, K))
x_est = np.zeros((n_rounds, K))

# Initialize estimates
q_hat[0, :] = 0.5
p_hat[0, :] = 0.5

# Reward Variables
x_reward = np.zeros((T, K))
X_obs = np.zeros((T, K))  # Observed response times (discrete)
Y_obs = np.zeros((T, K))  # Success indicators
F = np.zeros((T, K))      # Observed time (event or censoring)
f = np.zeros((T, K))      # Reward indicator (1 if event observed)
f_star = np.zeros((T, K)) # Optimal reward indicator
reward = np.zeros(T)
opt_reward = np.zeros(T)
regret = np.zeros(T)
cum_regret = np.zeros(T)

phi = np.zeros((n_rounds, K))
psi = np.zeros((n_rounds, K))

# INITIALIZATION PHASE
print("\nInitialization phase...")
for i in range(K):
    for step in range(init_steps_per_arm):
        if t1 >= T - 1:
            break
            
        x_reward[t1 + 1] = np.zeros(K)
        x_reward[t1 + 1, i] = B  # Give full budget to arm i during init

        # Sample from Discrete Weibull model
        Y_sample = rng.binomial(n=1, p=p_true)
        X_sample = np.array([discrete_weibull_sample(q_true[k], beta_vec[k], rng) for k in range(K)])
        
        X_obs[t1 + 1] = X_sample
        Y_obs[t1 + 1] = Y_sample

        for k in range(K):
            # Event observed if Y=1 and allocation >= response time
            if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                f[t1 + 1, k] = 1
                F[t1 + 1, k] = X_sample[k]
            else:
                f[t1 + 1, k] = 0
                F[t1 + 1, k] = x_reward[t1 + 1, k]  # Censored at allocation
                
            # Optimal policy reward
            if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                f_star[t1 + 1, k] = 1
            else:
                f_star[t1 + 1, k] = 0

        # Update estimates
        est_idx = t1 // K + 1
        if est_idx < n_rounds:
            phi[est_idx, i] = f[t1 + 1, i]
            psi[est_idx, i] = F[t1 + 1, i]
            update_estimates_discrete_weibull(
                t1 // K, i, beta_vec[i], phi, psi, n, sum_obs, q_hat, p_hat, x_est, B
            )
        
        # Compute regret
        compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)
        t1 += 1

# MAIN PHASE
print("Main phase...")
total_iterations = (T // K - 1 - t1 // K) * K
with tqdm(total=total_iterations, desc="Progress") as pbar:
    for t in range(t1 // K, T // K - 1):
        for i in range(K):
            if t1 >= T - 1:
                break
                
            # Sample from Discrete Weibull model
            Y_sample = rng.binomial(n=1, p=p_true)
            X_sample = np.array([discrete_weibull_sample(q_true[k], beta_vec[k], rng) for k in range(K)])

            # Compute confidence bounds
            p_bar, q_bar, q_bar_prime = compute_confidence_bounds_discrete_weibull_vec(
                t, i, p_hat, q_hat, n, x_est, B, D, K, beta_vec
            )
            
            # Select allocation using oracle with estimated bounds
            x_reward[t1 + 1] = select_allocation_discrete(q_bar, q_bar_prime, p_bar, B, beta_vec, method=method)
            x_reward[t1 + 1] = np.round(x_reward[t1 + 1]).astype(int)
            
            # Ensure sum = B
            diff = B - np.sum(x_reward[t1 + 1])
            if diff != 0:
                x_reward[t1 + 1, np.argmax(x_reward[t1 + 1])] += diff
            
            if t + 1 < n_rounds:
                x_est[t + 1, i] = x_reward[t1 + 1, i]

            X_obs[t1 + 1] = X_sample
            Y_obs[t1 + 1] = Y_sample

            for k in range(K):
                if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                    f[t1 + 1, k] = 1
                    F[t1 + 1, k] = X_sample[k]
                else:
                    f[t1 + 1, k] = 0
                    F[t1 + 1, k] = x_reward[t1 + 1, k]
                    
                if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                    f_star[t1 + 1, k] = 1
                else:
                    f_star[t1 + 1, k] = 0

            # Update estimates
            if t + 1 < n_rounds:
                phi[t + 1, i] = f[t1 + 1, i]
                psi[t + 1, i] = F[t1 + 1, i]
                update_estimates_discrete_weibull(
                    t, i, beta_vec[i], phi, psi, n, sum_obs, q_hat, p_hat, x_est, B
                )
            
            compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)
            t1 += 1
            pbar.update(1)

print(f"\nSimulation complete!")
actual_cum_regret = np.sum(regret[1:t1+1])
print(f"Final cumulative regret: {actual_cum_regret:.2f}")
print(f"Last index t1 = {t1}")

# DEBUG INFO
print("\n=== DEBUG INFO ===")
print(f"Total reward: {np.sum(reward[1:t1+1]):.2f}")
print(f"Total opt_reward: {np.sum(opt_reward[1:t1+1]):.2f}")
print(f"Mean instantaneous regret: {np.mean(regret[1:t1+1]):.4f}")
print(f"Std instantaneous regret: {np.std(regret[1:t1+1]):.4f}")

# Compare final allocation with optimal
print("\n=== ALLOCATION COMPARISON (last round) ===")
print(f"x_star = {x_star}")
print(f"x_reward (last) = {x_reward[t1].astype(int)}")
print(f"Allocation error (L1): {np.sum(np.abs(x_reward[t1] - x_star)):.0f}")

In [ ]:
# Plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Compute cumulative regret correctly
cum_regret_correct = np.cumsum(regret[1:t1+1])
t_axis = np.arange(1, t1+1)

# 1) Cumulative regret
ax1 = axes[0, 0]
ax1.plot(t_axis, cum_regret_correct, linewidth=2, label='Cumulative regret')
ax1.set_xlabel('t')
ax1.set_ylabel('Cumulative regret')
ax1.set_title('Cumulative regret of RA-UCB (Discrete Weibull)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2) Instantaneous regret (moving average)
ax2 = axes[0, 1]
window = min(100, len(regret[1:t1+1]) // 10)
if window > 1:
    regret_smooth = np.convolve(regret[1:t1+1], np.ones(window)/window, mode='valid')
    ax2.plot(np.arange(len(regret_smooth)), regret_smooth, linewidth=1, label=f'Instantaneous regret (MA {window})')
else:
    ax2.plot(t_axis, regret[1:t1+1], linewidth=1, label='Instantaneous regret')
ax2.axhline(0, color='k', linestyle='--', alpha=0.5)
ax2.set_xlabel('t')
ax2.set_ylabel('Instantaneous regret')
ax2.set_title('Instantaneous regret (moving average)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3) Mean reward per round
ax3 = axes[1, 0]
if window > 1:
    reward_smooth = np.convolve(reward[1:t1+1], np.ones(window)/window, mode='valid')
    opt_reward_smooth = np.convolve(opt_reward[1:t1+1], np.ones(window)/window, mode='valid')
    ax3.plot(np.arange(len(reward_smooth)), reward_smooth, label='Reward (algorithm)')
    ax3.plot(np.arange(len(opt_reward_smooth)), opt_reward_smooth, label='Reward (oracle)', linestyle='--')
else:
    ax3.plot(t_axis, reward[1:t1+1], label='Reward (algorithm)')
    ax3.plot(t_axis, opt_reward[1:t1+1], label='Reward (oracle)', linestyle='--')
ax3.set_xlabel('t')
ax3.set_ylabel('Reward')
ax3.set_title('Mean reward per round')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4) Allocation evolution for one arm
ax4 = axes[1, 1]
arm_to_track = 0
ax4.plot(t_axis, x_reward[1:t1+1, arm_to_track], label=f'x_reward[arm {arm_to_track}]', alpha=0.7)
ax4.axhline(x_star[arm_to_track], color='r', linestyle='--', label=f'x_star[arm {arm_to_track}]')
ax4.set_xlabel('t')
ax4.set_ylabel('Allocation')
ax4.set_title(f'Allocation to arm {arm_to_track} vs optimal')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()